# 1) 데이터 준비와 전처리

In [2]:
import os
import pandas as pd
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'moviehttps://8888-w23htc31avwr1yl2mf4pspevv.c2.prod.connect.ainize.ai/notebooks/aiffel/recommendata_iu/Movielens%20%EC%98%81%ED%99%94%20%EC%B6%94%EC%B2%9C.ipynb#_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [3]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [5]:
# rating 컬럼의 이름을 count로 바꿉니다.
ratings.rename(columns={'rating':'count'}, inplace=True)
ratings['count']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: count, Length: 836478, dtype: int64

In [6]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


사용하지 않는 컬럼 저장 후 데이터 프레임에서 삭제

In [10]:

# timestamp 따로 저장, ratings에서 삭제
timestamp = ratings['timestamp']
del ratings['timestamp']

# genre 따로 저장, movies에서 삭제
genre = movies['genre']
del movies['genre']

# 2) 분석해 봅시다.

- ratings에 있는 유니크한 영화 개수
- ratings에 있는 유니크한 사용자 수
- 가장 인기 있는 영화 30개(인기순)

In [11]:

# movie_id를 가지고 영화 제목 알아내기
# 영화 제목으로 movie_id 찾기

idx_to_movie = {}
movie_to_idx = {}

for i in range(movies.shape[0]):
    idx_to_movie[movies['movie_id'][i]] = movies['title'][i]
    movie_to_idx[movies['title'][i]] = movies['movie_id'][i]

print(idx_to_movie[300])
print(movie_to_idx['Quiz Show (1994)'])

Quiz Show (1994)
300


In [12]:

# movie_id == 300 확인. 일치!

movies[movies['movie_id']==300]

,movie_id,title
297,300,Quiz Show (1994)


In [13]:
# title == 'Quiz Show (1994)' 확인. 일치!

movies[movies['title']=='Quiz Show (1994)']

,movie_id,title
297,300,Quiz Show (1994)


In [14]:
# ratings에 있는 유니크한 영화 개수
print('유니크한 영화 개수는 :', ratings['movie_id'].nunique(), '개')

# rating에 있는 유니크한 사용자 수
print('유니크한 사용자 수는 :', ratings['user_id'].nunique(), '명')

유니크한 영화 개수는 : 3628 개
유니크한 사용자 수는 : 6039 명


In [15]:

# ratings 데이터 프레임에서 하나의 movie_id에 얼마나 많은 user_id가 있는지 합계 구한뒤
# 내림차순으로 정렬해서 30개만 보여주기

grouped = ratings['user_id'].groupby(ratings['movie_id']).sum()
top_30_movies_id = grouped.sort_values(ascending=False)[:30]
for i in range(len(top_30_movies_id)):
    print('가장 인기있는 {}순위 영화는: {}'.format(i+1, idx_to_movie[top_30_movies_id.index[i]]))

가장 인기있는 1순위 영화는: American Beauty (1999)
가장 인기있는 2순위 영화는: Star Wars: Episode IV - A New Hope (1977)
가장 인기있는 3순위 영화는: Star Wars: Episode V - The Empire Strikes Back (1980)
가장 인기있는 4순위 영화는: Star Wars: Episode VI - Return of the Jedi (1983)
가장 인기있는 5순위 영화는: Back to the Future (1985)
가장 인기있는 6순위 영화는: Saving Private Ryan (1998)
가장 인기있는 7순위 영화는: Silence of the Lambs, The (1991)
가장 인기있는 8순위 영화는: Raiders of the Lost Ark (1981)
가장 인기있는 9순위 영화는: Sixth Sense, The (1999)
가장 인기있는 10순위 영화는: Terminator 2: Judgment Day (1991)
가장 인기있는 11순위 영화는: Matrix, The (1999)
가장 인기있는 12순위 영화는: Fargo (1996)
가장 인기있는 13순위 영화는: Princess Bride, The (1987)
가장 인기있는 14순위 영화는: Jurassic Park (1993)
가장 인기있는 15순위 영화는: Godfather, The (1972)
가장 인기있는 16순위 영화는: Schindler's List (1993)
가장 인기있는 17순위 영화는: Men in Black (1997)
가장 인기있는 18순위 영화는: Shakespeare in Love (1998)
가장 인기있는 19순위 영화는: Shawshank Redemption, The (1994)
가장 인기있는 20순위 영화는: L.A. Confidential (1997)
가장 인기있는 21순위 영화는: Braveheart (1995)
가장 인기있는 22순위 영화는: E.T. the Extra-Terre

In [16]:
# 평점 기준으로 내림차순

ratings.sort_values(by=['count'], axis=0, ascending=False, inplace=True)
ratings.reset_index(drop=True, inplace=True)
ratings.head(10)

,user_id,movie_id,count
0,1,1193,5
1,2339,1952,5
2,4671,1210,5
3,2340,715,5
4,2340,2987,5
5,4671,380,5
6,1293,524,5
7,1293,527,5
8,2339,3929,5
9,2339,3922,5


In [17]:
# 평점이 높은 영화 30개

for i in range(30):
    print('가장 평점이 높은 영화 {}순위는 : {}'.format(i+1, movies[movies['movie_id'] == ratings.loc[i,'movie_id']].iloc[0,1]))

가장 평점이 높은 영화 1순위는 : One Flew Over the Cuckoo's Nest (1975)
가장 평점이 높은 영화 2순위는 : Midnight Cowboy (1969)
가장 평점이 높은 영화 3순위는 : Star Wars: Episode VI - Return of the Jedi (1983)
가장 평점이 높은 영화 4순위는 : Horseman on the Roof, The (Hussard sur le toit, Le) (1995)
가장 평점이 높은 영화 5순위는 : Who Framed Roger Rabbit? (1988)
가장 평점이 높은 영화 6순위는 : True Lies (1994)
가장 평점이 높은 영화 7순위는 : Rudy (1993)
가장 평점이 높은 영화 8순위는 : Schindler's List (1993)
가장 평점이 높은 영화 9순위는 : Bank Dick, The (1940)
가장 평점이 높은 영화 10순위는 : Bikini Beach (1964)
가장 평점이 높은 영화 11순위는 : Beach Party (1963)
가장 평점이 높은 영화 12순위는 : Fish Called Wanda, A (1988)
가장 평점이 높은 영화 13순위는 : Best in Show (2000)
가장 평점이 높은 영화 14순위는 : Phantom of the Opera, The (1943)
가장 평점이 높은 영화 15순위는 : Duck Soup (1933)
가장 평점이 높은 영화 16순위는 : Being John Malkovich (1999)
가장 평점이 높은 영화 17순위는 : Dirty Dancing (1987)
가장 평점이 높은 영화 18순위는 : Godfather: Part II, The (1974)
가장 평점이 높은 영화 19순위는 : Sleepless in Seattle (1993)
가장 평점이 높은 영화 20순위는 : Toy Story (1995)
가장 평점이 높은 영화 21순위는 : Lethal Weapon 4 (1998)
가장 평점

# 3) 내가 선호하는 영화를 5가지 골라서 ratings에 추가해 줍시다.

In [18]:
# Star Wars, Star Trek 영화 찾기

for i in range(movies.shape[0]):
    if movies.iloc[i,1][:9] == 'Star Wars':
        print(movies.iloc[i,1])
    elif movies.iloc[i,1][:9] == 'Star Trek':
        print(movies.iloc[i,1])

Star Wars: Episode IV - A New Hope (1977)
Star Trek: Generations (1994)
Star Wars: Episode V - The Empire Strikes Back (1980)
Star Wars: Episode VI - Return of the Jedi (1983)
Star Trek: First Contact (1996)
Star Trek: The Motion Picture (1979)
Star Trek VI: The Undiscovered Country (1991)
Star Trek V: The Final Frontier (1989)
Star Trek: The Wrath of Khan (1982)
Star Trek III: The Search for Spock (1984)
Star Trek IV: The Voyage Home (1986)
Star Trek: Insurrection (1998)
Star Wars: Episode I - The Phantom Menace (1999)


In [19]:
# 선호하는 영화 5개 리스트

my_favorite_title = ['Star Wars: Episode IV - A New Hope (1977)', 
               'Star Wars: Episode V - The Empire Strikes Back (1980)', 
               'Star Wars: Episode VI - Return of the Jedi (1983)',
              'Star Trek: Generations (1994)',
              'Star Trek: Insurrection (1998)']

# 선호하는 영화 movie_id 리스트 생성
my_favorite_id = []
for i in range(movies.shape[0]):
    if movies.iloc[i,1] in my_favorite_title:
        my_favorite_id.append(movies.iloc[i,0])

print(my_favorite_id)

[260, 329, 1196, 1210, 2393]


In [20]:
# 내 user_id = 기존 최대값 +1 = 6041

print(ratings['user_id'].max()) # 6040

6040


In [21]:
# user_id == 6041가 위 영화를 5번 봤다고 가정

my_playlist = pd.DataFrame({'user_id':[6041]*5, 'movie_id':my_favorite_id, 'count' : [5]*5 })

if not ratings.isin({'user_id':[6041]})['user_id'].any():
    ratings = ratings.append(my_playlist)
    print('추가!')
ratings[ratings['user_id'] == 6041]

추가!


,user_id,movie_id,count
0,6041,260,5
1,6041,329,5
2,6041,1196,5
3,6041,1210,5
4,6041,2393,5


In [22]:
ratings.reset_index(drop=False, inplace=True) # 인덱스 번호 초기화
ratings.drop('index', axis=1, inplace=True) # 'index' 컬럼 삭제
ratings.tail(10)

,user_id,movie_id,count
836473,4194,661,3
836474,1727,2581,3
836475,1727,174,3
836476,4194,1885,3
836477,3080,3536,3
836478,6041,260,5
836479,6041,329,5
836480,6041,1196,5
836481,6041,1210,5
836482,6041,2393,5


# 4) CSR matrix를 직접 만들어 봅시다.

In [23]:
# 실습 위에 설명보고 이해해서 만들어보기
from scipy.sparse import csr_matrix

num_user = ratings['user_id'].nunique()
num_movie = ratings['movie_id'].nunique()
# csr_data = csr_matrix((ratings['count'], (ratings.user_id, ratings.movie_id)), shape= (num_user, num_movie))
csr_data = csr_matrix((ratings['count'], (ratings.user_id, ratings.movie_id)))
csr_data

<6042x3953 sparse matrix of type '<class 'numpy.longlong'>'
	with 836483 stored elements in Compressed Sparse Row format>

In [24]:
# 수치 확인

print(num_user)
print(num_movie)
print(csr_data.shape)

6040
3628
(6042, 3953)


# 5) als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련시켜 봅시다.

In [25]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

In [26]:
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [27]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3953x6042 sparse matrix of type '<class 'numpy.longlong'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [28]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

# 6) 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보세요.

In [29]:
nyw, Star_Wars_4 = 6041, movie_to_idx['Star Wars: Episode IV - A New Hope (1977)']
nyw_vector, Star_Wars_4_vector = als_model.user_factors[nyw], als_model.item_factors[Star_Wars_4]

print('슝=3')

슝=3


In [30]:
nyw_vector

array([ 3.92936319e-01, -4.93862540e-01,  1.59450337e-01, -4.91313368e-01,
       -1.23987639e+00, -1.35395095e-01,  3.43875945e-01, -1.08716272e-01,
        4.17734206e-01,  3.80462289e-01, -5.58708251e-01, -6.88120186e-01,
        8.31876099e-02,  7.96296746e-02,  2.39274934e-01,  6.37536407e-01,
        5.64568520e-01,  9.89042938e-01, -4.41847920e-01,  7.48536214e-02,
        2.36556903e-01, -1.53343961e-01,  4.41697799e-02, -3.14266421e-02,
        1.05689816e-01,  4.93726760e-01,  9.11196411e-01, -1.51914759e-02,
       -5.06038487e-01, -4.01105732e-03,  9.25880432e-01,  4.48496610e-01,
        3.05600166e-01,  4.95758057e-01,  2.78200120e-01,  7.12572455e-01,
       -1.15028471e-01, -6.26353025e-01, -2.48325095e-01, -2.02178881e-02,
       -6.54517412e-01,  1.03380537e+00, -7.14733779e-01, -3.14839810e-01,
        2.38932595e-01,  1.48278937e-01,  3.80049080e-01, -1.64261442e-02,
       -5.72166622e-01,  7.36343041e-02, -1.44916109e-03, -1.09171212e-01,
        2.91117281e-01,  

In [31]:
Star_Wars_4_vector

array([ 0.01702679, -0.00045239,  0.03366721, -0.00420426, -0.0154751 ,
        0.01016093,  0.02869704, -0.00425661,  0.01646632, -0.01660776,
       -0.00628854, -0.00989012,  0.02392793, -0.00570932,  0.02780126,
        0.02555878,  0.01794959,  0.00967549, -0.02011838, -0.00789974,
        0.00746596,  0.01749115,  0.01793362, -0.01934217, -0.00682482,
        0.03584178,  0.00813533,  0.02742983, -0.00541453, -0.00506686,
        0.00216666,  0.02197314,  0.01829243,  0.03233269, -0.00273057,
        0.02440031, -0.01476694, -0.02663204, -0.01613159,  0.00936791,
       -0.01892746,  0.03278667,  0.00821151,  0.00029012,  0.03216586,
        0.02433718,  0.02625886,  0.00592758, -0.01846278,  0.02586852,
        0.00404573,  0.00806586,  0.03748239,  0.02174141, -0.00884002,
        0.00342112, -0.00978357, -0.00011902,  0.00421543,  0.01897224,
       -0.03258067,  0.03130841,  0.02362351, -0.00721357,  0.01013486,
       -0.03362592,  0.0173692 , -0.00139307,  0.04666349,  0.02

In [32]:
# nyw, Star_Wars_4_vector 내적하는 코드
np.dot(nyw_vector, Star_Wars_4_vector)

0.73252517

# 7) 내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다.
- AlternatingLeastSquares 클래스에 구현되어 있는 similar_items 메서드를 통하여 비슷한 영화 찾기

In [33]:
def get_similar_movie(movie_name: str):
    movie_id = movie_to_idx[movie_name]
    similar_movie = als_model.similar_items(movie_id)
    similar_movie = [idx_to_movie[i[0]] for i in similar_movie] # 위에서 정의한 idx_to_movie를 이용해 movie_id값을 이용해 제목 추출
    return similar_movie

In [34]:
get_similar_movie('Bug\'s Life, A (1998)') # 애니메이션들이 많이 추천되었다.

["Bug's Life, A (1998)",
 'Toy Story 2 (1999)',
 'Toy Story (1995)',
 'Aladdin (1992)',
 'Dinosaur (2000)',
 'Antz (1998)',
 'Iron Giant, The (1999)',
 'Beauty and the Beast (1991)',
 'Prince of Egypt, The (1998)',
 'Shakespeare in Love (1998)']

# 8) 내가 가장 좋아할 만한 영화들을 추천받아 봅시다.

In [35]:
user = 6041
# recommend에서는 user*item CSR Matrix를 받습니다.
movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
movie_recommended

[(2628, 0.5709095),
 (1356, 0.51579714),
 (1198, 0.48574233),
 (1372, 0.4504221),
 (1376, 0.41937506),
 (1197, 0.35596725),
 (1374, 0.34633598),
 (1375, 0.3435961),
 (1373, 0.3207959),
 (1371, 0.2910999),
 (1242, 0.24075991),
 (316, 0.23876092),
 (1291, 0.2036824),
 (2571, 0.20140573),
 (1240, 0.19993992),
 (2858, 0.19898367),
 (1214, 0.19002584),
 (1200, 0.18570688),
 (1270, 0.17663658),
 (912, 0.15806514)]

In [36]:
rec_list = [idx_to_movie[i[0]] for i in movie_recommended]
rec_list

['Star Wars: Episode I - The Phantom Menace (1999)',
 'Star Trek: First Contact (1996)',
 'Raiders of the Lost Ark (1981)',
 'Star Trek VI: The Undiscovered Country (1991)',
 'Star Trek IV: The Voyage Home (1986)',
 'Princess Bride, The (1987)',
 'Star Trek: The Wrath of Khan (1982)',
 'Star Trek III: The Search for Spock (1984)',
 'Star Trek V: The Final Frontier (1989)',
 'Star Trek: The Motion Picture (1979)',
 'Glory (1989)',
 'Stargate (1994)',
 'Indiana Jones and the Last Crusade (1989)',
 'Matrix, The (1999)',
 'Terminator, The (1984)',
 'American Beauty (1999)',
 'Alien (1979)',
 'Aliens (1986)',
 'Back to the Future (1985)',
 'Casablanca (1942)']

# 스타워즈 시리즈, 스타트랙 시리즈 들이 추천되었다